# Fairness Evaluation

 El desarrollo y uso responsable de modelos de inteligencia artificial es un tema de gran interés para los gobiernos, corporaciones y organizaciones internacionales ya que ésta se ha convertido en una herramienta de gran apoyo para una gran variedad de sectores. De acuerdo a [*Dignum, V.*](https://arxiv.org/abs/2205.10785) y [*Jobin, A., et. al.*](https://www.nature.com/articles/s42256-019-0088-2) las directrices éticas de las aplicaciones de la inteligencia artificial publicadas en más de 600 artículos por diversas organizaciones, convergen a cinco principios:
 * Transparencia: 
 
    Comprenden los esfuerzos para aumentar la explicabilidad, la interpretabilidad u otros actos de comunicación y divulgación. Se presenta como una forma de minimizar el daño y mejorar la IA, aunque algunas fuentes subrayan su beneficio por razones legales o para fomentar la confianza. Algunas fuentes también relacionan la transparencia con el diálogo, la participación y los principios de la democracia


 * Justicia y equidad:

   La justicia se expresa principalmente en términos de equidad y de prevención, control o mitigación de prejuicios y discriminación no deseados. Algunas fuentes se centran en la justicia como respeto a la diversidad, la inclusión y la igualdad, otras reclaman la posibilidad de apelar o impugnar las decisiones, o el derecho a la reparación y el remedio. Las fuentes también destacan la importancia de un acceso justo a la IA, a los datos y a los beneficios de la IA.


 * No maleficiencia: 

    Se refiere a la necesidad de evitar los daños y los riesgos, o de prevenir y mitigar los daños y los riesgos no deseados. Se afirma que la IA nunca debe causar un daño previsible o involuntario.

    
 * Responsabilidad:
 
   La responsabilidad rara vez se define. No obstante, las recomendaciones específicas incluyen actuar con "integridad" y aclarar la atribución de responsabilidad y la responsabilidad legal si es posible por adelantado, en los contratos


 * Privacidad:

    La IA ética considera la privacidad como un valor que hay que defender y como un derecho que hay que proteger. Aunque a menudo no se define, la privacidad se presenta a menudo en relación con la protección y seguridad de los datos.


El principio de equidad puede evaluarse de forma cuantitativa a través de la segmentación de ciertos grupos de interés y las métricas precisión y exahustividad. Con el fin de evaluar la posible existencia de sesgo de equidad en la clasificación xenófoba se considerará la segmentación por país de los tweets pertenecientes al conjunto de prueba.

In [1]:
#Imports

#Data reading and procesing
import pandas as pd
import numpy as np

#Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.io as pio
import plotly.express as px
pio.templates.default = "plotly_white"
from plotly.subplots import make_subplots

#Metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report as report

#scikit-learn metrics
from sklearn.metrics import classification_report

### Lectura de datos

Se cargará el archivo de datos `fairness_data.csv`, el cual contiene el texto de los tweets procesados, las etiquetas reales (xenófobos o no xenófobos), las etiquetas predichas por el modelo, así como el país de pertenencia de cada tweet. Si bien en este notebook no se utilizará la columna de texto, ésta fue utilizada para la predicción de las etiquetas. El conjunto de datos tal y como se propone tiene como objetivo evitar la reutilización de los códigos previamente utilizados para la predicción y procesamiento.

In [2]:
#read and show processed data
labeled_data = pd.read_csv('./assets/data/fairness_data.csv')
labeled_data.replace(np.nan, 'ND', inplace=True)
labeled_data.sort_values(by='xeno_proba')

,_id,text,date,label,predicted_xeno,xeno_probas,xeno_proba,a_code
1870,1343973985667723264,¡Inmigrantes! Ahora tenemos nuevamente campame...,2021-08-30,1,0,[0.50035353 0.49964647],0.500354,ND
1173,1353430950315778048,@usuario Y Fue un inmigrante.. hashtag sorry ...,2021-10-06,1,1,[0.4986857 0.5013143],0.501314,SLV
851,1149298465308106752,"@usuario No sea tan mentiroso Sr Bolivar, la H...",2021-11-30,1,0,[0.5021997 0.4978003],0.502200,PAN
633,1142978044300353536,@usuario Bobolonga la decisión de pedir x tipo...,2021-11-21,0,0,[0.50235867 0.49764133],0.502359,VEN
2011,1136778987353640960,@usuario La delincuencia aumenta a pasos galop...,2021-08-28,1,0,[0.50419359 0.49580641],0.504194,PER
...,...,...,...,...,...,...,...,...
2826,1150262759038361600,Pobres y migrantes en su propia patria - Proce...,2021-11-11,0,0,[9.99942706e-01 5.72944453e-05],0.999943,MEX
4037,1140036161836134400,"Crisis en Venezuela: el ""récord"" de migrantes ...",2021-09-30,0,0,[9.99942927e-01 5.70726665e-05],0.999943,MEX
3537,1195171098964430848,La huella digital de los migrantes venezolanos...,2021-11-25,0,0,[9.99944563e-01 5.54373318e-05],0.999945,VEN
4179,1227333408881283072,Costo de la vida y limbo migratorio: Principal...,2021-10-17,0,0,[9.99946447e-01 5.35527554e-05],0.999946,CHL


### Distribución porcentual de tweets por país

Se procede a calcular la distribución de tweets por país y etiqueta, para lo cual se utilizará la función `groupby()` de la librería `pandas` para obtener el conteo de tweets por país. Se realizará un gráfico de barras para visualizar la distribución de tweets por país, esto con el fin de identificar la cantidad de muestras de evaluación disponibles.

In [3]:
#data frame with label counts by country and type
label_counts_country = labeled_data.groupby(['a_code']).label.value_counts().to_frame()
label_counts_country.rename(columns={'label':'# Etiquetas'}, inplace=True)
label_counts_country.reset_index(inplace=True)
label_counts_country.rename(columns={'a_code':'País', 'label':'Tipo'}, inplace=True)
label_counts_country.Tipo.replace({0:'No xenófobo', 1:'Xenófobo'}, inplace=True)

In [4]:
#data visualization
fig = px.bar(label_counts_country.sort_values(by=['# Etiquetas'], ascending=False), x='País', y='# Etiquetas', color='Tipo', text_auto='percent', barmode='group', 
    title='Distribución de etiquetas por país y tipo en el conjunto de prueba', color_discrete_map={'No xenófobo':'#1f77b4', 'Xenófobo':'#FF2626'},
    width=1400, height=800)
fig.update_traces(textposition='outside')
fig.show()

### Evaluación de métricas por país

La evaluación de equidad requiere únicamente de obtener las métricas evaluación del modelo por cada sector. Para este caso los sectores de interés serán los países de origen de los tweets. 

Diremos que un modelo es equitativo si las métricas de evaluación son similares para todos los sectores.

Calculemos ahora las métricas de evaluación para el país Colombia. Posteriormente se generará una función que permita calcular las métricas de evaluación para todos los países.

In [6]:
#Get data from Colombia
colombia_data = labeled_data[labeled_data.a_code == 'COL']
colombia_data.reset_index(inplace=True, drop=True)

#print metrics for Colombia data through classification_report function
print(classification_report(y_true = colombia_data.label.to_list(), 
                                y_pred = colombia_data.predicted_xeno.to_list(),
                                target_names=['No xenófobo', 'Xenófobo'], digits=4))

              precision    recall  f1-score   support

 No xenófobo     0.8886    0.9335    0.9105       316
    Xenófobo     0.7717    0.6574    0.7100       108

    accuracy                         0.8632       424
   macro avg     0.8301    0.7955    0.8102       424
weighted avg     0.8588    0.8632    0.8594       424



Construyamos ahora una función que permita calcular e imprimir las métricas de evaluación para un conjunto de países. Este conjunto puede ser definido por el usuario.

In [7]:
def print_report(test_data, country_list):
    """_summary_: Function to print metrics for each country in a list

    Args:
        test_data (pd.DataFrame): data frame with test labels, predictions and country codes
        country_list (list): list of country codes to print metrics for
    """
    for country in country_list:
        country_data = test_data[test_data.a_code == country]
        country_data.reset_index(inplace=True, drop=True)
        print('Código del país: ', country)
        print(classification_report(y_true = country_data.label.to_list(), 
                                y_pred = country_data.predicted_xeno.to_list(),
                                target_names=['No xenófobo', 'Xenófobo'], digits=4))
        print('-'*60)

In [12]:
print_report(labeled_data, ['ND', 'ECU', 'ARG', 'COL', 'CHL'])

Código del país:  ND
              precision    recall  f1-score   support

 No xenófobo     0.9220    0.8931    0.9073      1244
    Xenófobo     0.7551    0.8135    0.7832       504

    accuracy                         0.8701      1748
   macro avg     0.8385    0.8533    0.8452      1748
weighted avg     0.8739    0.8701    0.8715      1748

------------------------------------------------------------
Código del país:  ECU
              precision    recall  f1-score   support

 No xenófobo     0.9479    0.9579    0.9529        95
    Xenófobo     0.8400    0.8077    0.8235        26

    accuracy                         0.9256       121
   macro avg     0.8940    0.8828    0.8882       121
weighted avg     0.9247    0.9256    0.9251       121

------------------------------------------------------------
Código del país:  ARG
              precision    recall  f1-score   support

 No xenófobo     0.9286    0.8994    0.9137       159
    Xenófobo     0.4074    0.5000    0.4490       

Las métricas de evaluación para cada país se muestran en la celda superior, donde notamos poca variación con respecto al valor-F de la clase no xenófoba. Sin embargo, la variación llega a ser significativa para la clase xenófoba. Esto puede deberse a que la cantidad de tweets xenófobos en el conjunto de prueba es muy pequeña para algunos países como Argentina y Ecuador. Ambos ejemplos ofrecen polos opuestos; parra argentina los valores de precisión y exhaustividad para la clase xenófoba son muy bajos, mientras que para Ecuador los valores son altos. Una posible respuesta a esta problemática está relacionada con la cantidad de etiquetas de clase xenófoba, ya que se podrían sobre- y des-estimar las métricas de evaluación.
Notemos que para los países con mayor cantidad de muestras de la clase xenófoba; Chile, Colombia y ND las métricas de evaluación son similares.
Aún con estas similitudes, las pequeñas variaciones muestran que el modelo presenta una pequeña inequidad. Lo cual podría deberse a que el modelo fue entrenado con un conjunto de datos que no estaba distribuida uniformemente por país generandose sesgos de equidad.

Parra ajustar la equidad del modelo deben agragarse más muestras etiquetadas de ambas clases para los países con menor cantidad de muestras a los conjuntos de entrenamiento y prueba. Habiendo conocido el método de active learning, podría buscarse una combianación de ambos para mejorar el modelo en general y la equidad en particular.

